##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# 训练你的第一个神经网络: 基本分类

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://tensorflow.google.cn/tutorials/keras/text_classification"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png" />在 tensorFlow.google.cn 上查看</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/zh-cn/tutorials/keras/text_classification.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png" />在 Google Colab 中运行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/zh-cn/tutorials/keras/text_classification.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png" />在 GitHub 上查看源代码</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/zh-cn/tutorials/keras/text_classification.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png" />下载 notebook</a>
  </td>
</table>

Note: 我们的 TensorFlow 社区翻译了这些文档。因为社区翻译是尽力而为， 所以无法保证它们是最准确的，并且反映了最新的
[官方英文文档](https://www.tensorflow.org/?hl=en)。如果您有改进此翻译的建议， 请提交 pull request 到
[tensorflow/docs](https://github.com/tensorflow/docs) GitHub 仓库。要志愿地撰写或者审核译文，请加入
[docs-zh-cn@tensorflow.org Google Group](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-zh-cn)。

这个指南训练了一个神经网络模型去对衣服的图形进行分类，例如运动鞋和衬衫。您不懂所有细节也无妨。这是一个对于完整TensorFlow的项目的快速入门，其中细节会在需要的时候解释。

这个指南使用了 [tf.keras](https://www.tensorflow.org/guide/keras), 一个 TensorFlow 中构建和训练模型的高级接口。

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## 导入Fashion MNIST数据集

这个指南使用了包含 10 种类，70000 张灰度图像的 [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) 数据集。这些图片展示了低分辨率的个人衣物（28x28像素），如图所示：

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST samples</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>

Fashion MNIST 是用来当作传统的 [MNIST](http://yann.lecun.com/exdb/mnist/) 数据集的简单替代。MNIST 数据集对于计算机视觉的机器学习项目就如同 "Hello, World" 一样。MNIST 数据集是包含了手写的数字图像（0,1,2等等），和这里我们用到的服装图像的格式一样。

这个指南使用　Fashion MNIST 进行多样化，因为它比传统的 MNIST 更稍微有挑战性。两者的数据集都相对较小，用于验证算法是否如预期一般。他们是测试和调试代码的优良起点。

共有 60000 张图片用于训练模型，10000 张图片用于验证网络模型学习分类图片的精度。您可以直接从 TensorFlow 加载 Fashion MNIST ，直接导入并加载数据：

In [0]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

加载数据集，返回四个 NumPy 数组：

* `train_images` 和 `train_labels` 数组是  *训练集*，即模型用来学习的数据。
* 模型通过 *test set* 来测试， 即 `test_images`, 和 `test_labels` 数组。

图像是28x28的NumPy的数组，取值从0到255。  *labels* 是从0到9的整数数组。 这些对应了图像所代表的衣服种类：

<table>
  <tr>
    <th>Label</th>
    <th>Class</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td>
  </tr>
</table>

每一张图片都打上了唯一的标签，因为类别名没有包括在数据集中，所以将它们存在这里以便之后绘制图形：

In [0]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## 探索数据

让我们在训练模型之前探索数据集的格式。下面说明了数据集中有 60000 张图，每张图片以 28*28像素显示：

In [0]:
train_images.shape

类似的，数据集中有 60000 个标签：

In [0]:
len(train_labels)

每个标签都是 0 到 9 的整数：

In [0]:
train_labels

测试集中 10000 张图片。类似的，每张图片都以 28 x 28 像素呈现：

In [0]:
test_images.shape

测试集中包含 10000 个图像标签：

In [0]:
len(test_labels)

## 处理数据

训练网络之前必须预处理数据。如果您检查训练集里的第一张图片，您会发现像素值在 0 到 255 之间：

In [0]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

在投入神经网络模型训练前将这些值放缩到 0 和 1 之间。因此，我们把数值都除以 255 。对*训练集*和*测试集*进行相同操作是很重要的：

In [0]:
train_images = train_images / 255.0

test_images = test_images / 255.0

为了验证数据格式是否正确，同时准备好构建和训练网络，让我们从训练集中显示前25个图像，并在其下方显示种类名。

In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## 构建模型

构建神经网络需要确认模型的层，然后编译网络。

### 设置网络层

神经网络最基本的组成就是*网络层*. 网络层从提供的数据中提取表示， 这些表示是为了让当前的问题更加有意义。

大多数深度学习是由串连在一起的网络层所组成。大多数网络层，例如`tf.keras.layers.Dense`, 有训练时学习到的参数。

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

在网络中的第一层，tf.keras.layers.Flatten`, 将图像从二维数组（28x28个像素）转换为一维数组（28x28 = 784个像素）。将这个网络层视为它将图像中未堆叠的像素排列在一起。这个网络层没有需要学习的参数;它仅仅把数据格式化。

在像素被展平之后，网络由一个包含有两个`tf.keras.layers.Dense` 层. 他们是稠密链接层或全连接层。第一个 `Dense`层有 128 个节点 (或神经元)。 第二层 (最后一层)是十节点的*softmax* 层，它将会返回十个总和为一的概率数字的数组. 每个节点包含一个分数，表示当前图像属于10个类别之一的概率。

### 编译模型

在模型训练之前，需要数个设置。以下需要在 *编译* 步骤添加:

* *损失函数* —这衡量了模型在训练中的准确程度。您应该让这个函数最小化来驱动模型往正确的方向拟合。
* *优化器* —这是模型根据其所知的数据和损失函数更新的方式。
* *评价方式* —用来检测训练和测试的步骤。下面的例子使用*准确率*，即正确分类的图像的比例。

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## 训练模型

通过以下步骤来训练神经网络模型：

1. 给模型提供训练数据。在这个例子里，训练数据在 `train_images` 和  `train_labels`  数组。
2. 模型学习去匹配图像和标签
3. 您使用模型对测试集进行预测，这个例子中为 `test_images`  数组。要验证预测结果是否匹配 `test_labels` 数组中的标签。

通过调用 `model.fit` 方法，模型对训练数据拟合，来开始训练：

In [0]:
model.fit(train_images, train_labels, epochs=5)

随着模型的训练，损失和准确率等都会显示出来。模型再训练数据上达到了约0.88（88%）的准确率。

## 评估准确率

下一步，在测试集上对比模型的表现：

In [0]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)

事实证明,在测试集上准确率低于在训练集上的准确性。训练准确性和测试准确性的差异来源于*过拟合*。*过拟合*是机器学习的模型在新数据上表现的比训练数据差。

## 开始预测

通过训练好的模型，您可以使用它对图像进行预测。

In [0]:
predictions = model.predict(test_images)

这里，模型已经对测试集中每一张图片进行了预测。让我们一个看看第一个结果​：

In [0]:
predictions[0]

结果是包含十个数字的数组。他们代表了模型对10种衣服的信任度。我们可以看到哪种标签的置信度最高：

In [0]:
np.argmax(predictions[0])

所以模型认为图片最有可能是 ankle boot, 或者`class_names[9]`。通过检查测试标签可以验证结果是否正确​：

In [0]:
test_labels[0]

我们将十个种类的预测集的图像都画出来：.

In [0]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  plt.imshow(img, cmap=plt.cm.binary)
  
  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'
  
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)
  
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

让我们看看第零张图片，预测和预测数组。

In [0]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [0]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

让我们吧这些图片和他们的预测值画出来。 正确的预测标签是蓝色的，错误的是红色的。数字给出了预测标签的百分比（最高100）。 请注意即使模型很自信也可能会预测错。

In [0]:
# 绘制前X个预测图像，预测标签和真实标签。
# 正确预测为蓝，错误预测为红。
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.show()

最后，使用训练好的模型对单个图像进行预测。

In [0]:
# 从测试集中获取数据。
img = test_images[1]

print(img.shape)

`tf.keras` 模型在优化后可以一次性的对批量或者一个集合的模型进行预测。所以，即使只用一张图片，也要将其添加到列表里面：

In [0]:
# 把图片作为唯一成员添加到这一批次中。
img = (np.expand_dims(img,0))

print(img.shape)

现在来预测图像：

In [0]:
predictions_single = model.predict(img)

print(predictions_single)

In [0]:
plot_value_array(1, predictions_single[0], test_labels)
plt.xticks(range(10), class_names, rotation=45)
plt.show()

`model.predict` 返回一个有着列表的列表，每个列表的数据对应一张图像。获取在这一批中我们仅有的图像的预测值:

In [0]:
prediction_result = np.argmax(predictions_single[0])
print(prediction_result)

模型预测的标签是2。